Виконання алгоритму будемо писати на мові python, та використаємо бібліотеку pandas, яка є гнучкою та багатофункціональною для роботи з різними масивами даних.
Pandas оптимізований для векторизованих операцій, що значно швидше ніж поелементні операції 
Вбудована підтримка для обробки missing values
Потужні функції для групування, агрегації та трансформації даних
Хороші можливості для візуалізації через інтеграцію з matplotlib/seaborn
Багата екосистема для машинного навчання та аналізу даних
Хороша підтримка для роботи з різними форматами файлів (Excel, CSV, JSON тощо)

In [2]:
import pandas as pd
import numpy as np

Щоб завантажувати дані з SellingPartnerApi в пайтоні можна скористатись бібліотеками requests або aiohttp. requests простіша в написанні , а aiohttp краще використовувати для роботи з великими даними так як там операції виконуються асинхронно.

In [ ]:
import datetime
import time
import requests

# Параметри для API
id_amz_selling_partner_account = "your_account_id"
id_amz_marketplace = "your_marketplace_id"
access_token = "your_access_token"
report_type = "GET_BRAND_ANALYTICS_SEARCH_TERMS_REPORT"

# Визначення періоду звіту (наприклад, за день)
report_period = "DAY"  # Можна змінити на WEEK, MONTH або QUARTER
date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")  # Для DAY

# Визначення dataStartTime та dataEndTime відповідно до reportPeriod
if report_period == "DAY":
    data_start_time = f"{date}T00:00:01+00:00"
    data_end_time = f"{date}T23:59:59+00:00"
elif report_period == "WEEK":
    # Для WEEK dataStartTime має бути неділя, а dataEndTime - субота
    start_of_week = (datetime.datetime.now() - datetime.timedelta(days=datetime.datetime.now().weekday() + 1)).strftime("%Y-%m-%d")
    end_of_week = (datetime.datetime.now() + datetime.timedelta(days=(5 - datetime.datetime.now().weekday()))).strftime("%Y-%m-%d")
    data_start_time = f"{start_of_week}T00:00:01+00:00"
    data_end_time = f"{end_of_week}T23:59:59+00:00"
elif report_period == "MONTH":
    # Для MONTH dataStartTime має бути перший день місяця, а dataEndTime - останній
    first_day_of_month = datetime.datetime.now().replace(day=1).strftime("%Y-%m-%d")
    last_day_of_month = (datetime.datetime.now().replace(day=1) + datetime.timedelta(days=32)).replace(day=1) - datetime.timedelta(days=1)
    data_start_time = f"{first_day_of_month}T00:00:01+00:00"
    data_end_time = f"{last_day_of_month.strftime('%Y-%m-%d')}T23:59:59+00:00"
elif report_period == "QUARTER":
    # Для QUARTER dataStartTime має бути перший день кварталу, а dataEndTime - останній
    current_month = datetime.datetime.now().month
    if current_month in [1, 2, 3]:
        first_day_of_quarter = datetime.datetime.now().replace(month=1, day=1).strftime("%Y-%m-%d")
        last_day_of_quarter = datetime.datetime.now().replace(month=3, day=31).strftime("%Y-%m-%d")
    elif current_month in [4, 5, 6]:
        first_day_of_quarter = datetime.datetime.now().replace(month=4, day=1).strftime("%Y-%m-%d")
        last_day_of_quarter = datetime.datetime.now().replace(month=6, day=30).strftime("%Y-%m-%d")
    elif current_month in [7, 8, 9]:
        first_day_of_quarter = datetime.datetime.now().replace(month=7, day=1).strftime("%Y-%m-%d")
        last_day_of_quarter = datetime.datetime.now().replace(month=9, day=30).strftime("%Y-%m-%d")
    else:
        first_day_of_quarter = datetime.datetime.now().replace(month=10, day=1).strftime("%Y-%m-%d")
        last_day_of_quarter = datetime.datetime.now().replace(month=12, day=31).strftime("%Y-%m-%d")
    data_start_time = f"{first_day_of_quarter}T00:00:01+00:00"
    data_end_time = f"{last_day_of_quarter}T23:59:59+00:00"
else:
    raise ValueError("Невірний reportPeriod. Допустимі значення: DAY, WEEK, MONTH, QUARTER.")

# Створення запиту на формування звіту
url = "https://sellingpartnerapi.amazon.com/reports"
payload = {
    "reportType": report_type,
    "dataStartTime": data_start_time,
    "dataEndTime": data_end_time,
    "marketplaceIds": [id_amz_marketplace],
    "reportOptions": {
        "reportPeriod": report_period
    }
}
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
response = requests.post(url, json=payload, headers=headers)

# Перевірка наявності помилок у відповіді
if response.status_code != 200:
    raise Exception(f"Помилка при створенні звіту: {response.status_code} - {response.text}")

report_id = response.json()["reportId"]

# Очікування формування звіту
max_retries = 10
retry_delay = 10  # Початкова затримка в секундах

for attempt in range(max_retries):
    report_status_url = f"https://sellingpartnerapi.amazon.com/reports/{report_id}"
    status_response = requests.get(report_status_url, headers=headers)

    # Перевірка наявності помилок у відповіді
    if status_response.status_code != 200:
        raise Exception(f"Помилка при перевірці статусу звіту: {status_response.status_code} - {status_response.text}")

    processing_status = status_response.json()["processingStatus"]

    if processing_status == "DONE":
        # Завантаження звіту
        report_document_id = status_response.json()["reportDocumentId"]
        report_document_url = f"https://sellingpartnerapi.amazon.com/reports/{report_document_id}"
        report_data_response = requests.get(report_document_url, headers=headers)

        # Перевірка наявності помилок у відповіді
        if report_data_response.status_code != 200:
            raise Exception(f"Помилка при завантаженні звіту: {report_data_response.status_code} - {report_data_response.text}")

        report_data = report_data_response.json()

        # Фільтрація даних за departmentName == 'Amazon.com'
        filtered_data = [item for item in report_data["reportData"] if item["departmentName"] == "Amazon.com"]
        break
    else:
        time.sleep(retry_delay * (attempt + 1))
else:
    raise Exception("Не вдалося отримати звіт після максимальної кількості спроб.")



In [ ]:
import aiohttp
import asyncio
import datetime

async def fetch_report():
    async with aiohttp.ClientSession() as session:
        # Визначення дати
        date = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime("%Y-%m-%d")

        # Параметри запиту
        payload = {
            "reportType": "GET_BRAND_ANALYTICS_SEARCH_TERMS_REPORT",
            "dataStartTime": f"{date}T00:00:01+00:00",
            "dataEndTime": f"{date}T23:59:59+00:00",
            "marketplaceIds": ["your_marketplace_id"]
        }
        headers = {
            "Authorization": "Bearer your_access_token",
            "Content-Type": "application/json"
        }

        # Створення запиту на формування звіту
        async with session.post("https://sellingpartnerapi.amazon.com/reports", json=payload, headers=headers) as response:
            if response.status != 200:
                raise Exception(f"Помилка при створенні звіту: {response.status} - {await response.text()}")
            report_id = (await response.json())["reportId"]

        # Очікування формування звіту
        max_retries = 10
        retry_delay = 10

        for attempt in range(max_retries):
            async with session.get(f"https://sellingpartnerapi.amazon.com/reports/{report_id}", headers=headers) as status_response:
                if status_response.status != 200:
                    raise Exception(f"Помилка при перевірці статусу звіту: {status_response.status} - {await status_response.text()}")

                processing_status = (await status_response.json())["processingStatus"]

                if processing_status == "DONE":
                    report_document_id = (await status_response.json())["reportDocumentId"]
                    async with session.get(f"https://sellingpartnerapi.amazon.com/reports/{report_document_id}", headers=headers) as report_data_response:
                        if report_data_response.status != 200:
                            raise Exception(f"Помилка при завантаженні звіту: {report_data_response.status} - {await report_data_response.text()}")

                        report_data = await report_data_response.json()

                        # Фільтрація даних за departmentName == 'Amazon.com'
                        filtered_data = [item for item in report_data["reportData"] if item["departmentName"] == "Amazon.com"]
                        return filtered_data
                else:
                    await asyncio.sleep(retry_delay * (attempt + 1))

        raise Exception("Не вдалося отримати звіт після максимальної кількості спроб.")

# Запуск асинхронної функції
filtered_data = asyncio.run(fetch_report())


In [ ]:
# зберігати можна в різних форматах, для прикладу json:
import json

# Збереження filtered_data у JSON файл
with open("filtered_data.json", "w", encoding="utf-8") as file:
    json.dump(filtered_data, file, ensure_ascii=False, indent=4)

print("Дані збережено у файл filtered_data.json")

Перейдемо до виконання безпосередньо нашого завдання. Розіб"ємо це на дві частини. 1 - виконання згідно алгоритму, 2 -власне бачення даної задачі.
Частина 1 : згідно алгоритму
Завантажуємо дані з файлу excel. Наприклад з файлу 13.xls та подивимось як виглядає верхівка таблиці

In [3]:

# Прочитати Excel-файл
df = pd.read_excel(r'C:\Users\user\Downloads\brandAnalytics\data7days\13.xls', skiprows=1, header=0)

# Показати перші кілька рядків DataFrame
print(df.head())

   Department                Search Term  Search Frequency Rank  \
0  Amazon.com      christmas decorations                      1   
1  Amazon.com  thanksgiving outfit women                      2   
2  Amazon.com                yellowstone                      3   
3  Amazon.com       yellowstone season 5                      4   
4  Amazon.com             christmas tree                      5   

  #1 Clicked ASIN                                   #1 Product Title  \
0      B08B5PHMH6  Ivenf Christmas Decorations Outdoor Yard Front...   
1      B09FXHL254  SheIn Women's V Neck Sleeveless Pocket Corduro...   
2      B0B8T22BYG                      Yellowstone: Inside the Ranch   
3      B0B8T22BYG                      Yellowstone: Inside the Ranch   
4      B09F6GYVZL  Yaheetech 4.5ft Pre-lit Artificial Christmas T...   

   #1 Click Share  #1 Conversion Share #2 Clicked ASIN  \
0        0.052653             0.054300      B0B652CTLV   
1        0.045553             0.019108      B0BF

In [84]:
#перевіримо типи даних значень таблиці
df.dtypes

Department                object
Search Term               object
Search Frequency Rank      int64
#1 Clicked ASIN           object
#1 Product Title          object
#1 Click Share           float64
#1 Conversion Share      float64
#2 Clicked ASIN           object
#2 Product Title          object
#2 Click Share           float64
#2 Conversion Share      float64
#3 Clicked ASIN           object
#3 Product Title          object
#3 Click Share           float64
#3 Conversion Share      float64
dtype: object

In [136]:
# Для кращої читабельності переведемо значення в відсотки(опціонально)
percent_columns = ['#1 Click Share', '#1 Conversion Share', '#2 Click Share', '#2 Conversion Share', '#3 Click Share', '#3 Conversion Share']

# Помножити значення у відсоткових стовпцях на 100
df[percent_columns] = df[percent_columns] * 100

# Показати перші кілька рядків DataFrame
print(df.head())

   Department                Search Term  Search Frequency Rank  \
0  Amazon.com      christmas decorations                      1   
1  Amazon.com  thanksgiving outfit women                      2   
2  Amazon.com                yellowstone                      3   
3  Amazon.com       yellowstone season 5                      4   
4  Amazon.com             christmas tree                      5   

  #1 Clicked ASIN                                   #1 Product Title  \
0      B08B5PHMH6  Ivenf Christmas Decorations Outdoor Yard Front...   
1      B09FXHL254  SheIn Women's V Neck Sleeveless Pocket Corduro...   
2      B0B8T22BYG                      Yellowstone: Inside the Ranch   
3      B0B8T22BYG                      Yellowstone: Inside the Ranch   
4      B09F6GYVZL  Yaheetech 4.5ft Pre-lit Artificial Christmas T...   

   #1 Click Share  #1 Conversion Share #2 Clicked ASIN  \
0          5.2653               5.4300      B0B652CTLV   
1          4.5553               1.9108      B0BF

Також переіменуємо деякі стовпці

In [137]:
df.columns

Index(['Department', 'Search Term', 'Search Frequency Rank', '#1 Clicked ASIN',
       '#1 Product Title', '#1 Click Share', '#1 Conversion Share',
       '#2 Clicked ASIN', '#2 Product Title', '#2 Click Share',
       '#2 Conversion Share', '#3 Clicked ASIN', '#3 Product Title',
       '#3 Click Share', '#3 Conversion Share'],
      dtype='object')

In [138]:
new_columns = ['Department', 'Search_Term', 'Search_Frequency_Rank', 'ASIN_1',
       'Product_Title_1', 'Click_Share_1', 'Conversion_Share_1',
       'ASIN_2', 'Product_Title_2', 'Click_Share_2', 'Conversion_Share_2',
        'ASIN_3', 'Product_Title_3', 'Click_Share_3', 'Conversion_Share_3',]
df.columns = new_columns
df.columns

Index(['Department', 'Search_Term', 'Search_Frequency_Rank', 'ASIN_1',
       'Product_Title_1', 'Click_Share_1', 'Conversion_Share_1', 'ASIN_2',
       'Product_Title_2', 'Click_Share_2', 'Conversion_Share_2', 'ASIN_3',
       'Product_Title_3', 'Click_Share_3', 'Conversion_Share_3'],
      dtype='object')

In [139]:
df.shape
# маємо 317 рядків та 15 стовпців

(317, 15)

Перевіримо чи немає пустих значень (NaN) в таблиці

In [140]:
df.isnull().sum()

Department               0
Search_Term              0
Search_Frequency_Rank    0
ASIN_1                   0
Product_Title_1          0
Click_Share_1            0
Conversion_Share_1       0
ASIN_2                   0
Product_Title_2          0
Click_Share_2            0
Conversion_Share_2       0
ASIN_3                   0
Product_Title_3          0
Click_Share_3            0
Conversion_Share_3       0
dtype: int64

In [141]:
#Подивимось на  перші кілька рядків DataFrame
df.head()

,Department,Search_Term,Search_Frequency_Rank,ASIN_1,Product_Title_1,Click_Share_1,Conversion_Share_1,ASIN_2,Product_Title_2,Click_Share_2,Conversion_Share_2,ASIN_3,Product_Title_3,Click_Share_3,Conversion_Share_3
0,Amazon.com,christmas decorations,1,B08B5PHMH6,Ivenf Christmas Decorations Outdoor Yard Front...,5.2653,5.4300,B0B652CTLV,318 PCS 12 Sheets Christmas Snowflake Window C...,3.9585,6.5093,B07HHBQJGJ,LUDILO 135Pcs Christmas Window Clings Snowflak...,3.7396,7.5548
1,Amazon.com,thanksgiving outfit women,2,B09FXHL254,SheIn Women's V Neck Sleeveless Pocket Corduro...,4.5553,1.9108,B0BFWXHD2F,Gobble Funny Thanksgiving T-Shirt Women Turkey...,1.3965,2.1231,B0BFWVCM64,Gobble Funny Thanksgiving T-Shirt Women Turkey...,1.1762,1.5924
2,Amazon.com,yellowstone,3,B0B8T22BYG,Yellowstone: Inside the Ranch,37.6087,64.6818,B0B8QQK2TG,Yellowstone Season 5: Pts 1 & 2,12.2555,3.6922,B07DWNHLFG,Daybreak,7.6955,5.8172
3,Amazon.com,yellowstone season 5,4,B0B8T22BYG,Yellowstone: Inside the Ranch,47.0374,74.0830,B0B8QQK2TG,Yellowstone Season 5: Pts 1 & 2,18.3375,5.1684,B0B8Q96Q59,Yellowstone Season 5: Pts 1 & 2,14.7028,8.3781
4,Amazon.com,christmas tree,5,B09F6GYVZL,Yaheetech 4.5ft Pre-lit Artificial Christmas T...,4.2566,2.8793,B08J7JRJL3,Best Choice Products 6ft Pre-Lit Pre-Decorated...,4.1603,5.2102,B0BHTTZ9H9,Best Choice Products 4.5ft Pre-Lit Snow Flocke...,3.8411,3.7934


In [363]:
# зробимо копію нашого датафрейму
new_df = df.copy()

new_df.shape  # Розмірність 



(317, 15)

За допомогою функції calculate_sampling_orders шукаємо варіанти кількості продажів для кожного асіна та загальної кількості продажів для окремого рядка (в даному випадку ми це робимо емпіричним шляхом будуючи n комбінацій)
В функції prepare_data_samples ми проходимось по кожному рядку застосовуємо calculate_sampling_orders беремо 5 перших зразків та обраховуємо загальну кількість кліків та перевіряємо умову (Для searchTerms що складаються з одного слова ставимо обмеження, що конверсія має бути меньше 2%). Створюємо  новий масив.
ps я не погоджуюсь з автором щодо цього твердження: "Тут ще є момент коли ми шукаємо можливі варіанти конверсії (яка дорівнює 100% / на можливий варіант продажів)" 
В даному випадку, конверсія це відношення кількості продажів(замовлень) до кількості запитів та помножена на 100 щоб виражатись в відсотках. Щоб знайти загальну конверсію потрібно знайти total orders(загальна кількість замовлень) та searchVolume(загальна кількість запитів) а потім знайти відношення

In [368]:

def calculate_sampling_orders(row: pd.Series, DELTA = 0.05) -> pd.DataFrame:
    """Розраховує можливі приклади замовлень на основі часток конверсій."""
    conversions = list(filter(None, [row['Conversion_Share_1'], row['Conversion_Share_2'], row['Conversion_Share_3']]))
    # Перевіряємо, чи список порожній
    if not conversions:
        return pd.DataFrame()  
    # Обчислюємо min_conversion
    min_conversion = min(conversions)
    # Обчислюємо можливі замовлення
    
    # Створюємо порожній DataFrame з визначеними стовпцями
    columns = ['a1', 'a2', 'a3', 'others', 'totalOrders']
    results_df = pd.DataFrame(columns=columns)
    
    for i in range(10, 200, 40):
        nsd = min_conversion / i
        a1 = round(row['Conversion_Share_1'] / nsd, 2)
        a2 = round(row['Conversion_Share_2'] / nsd, 2)
        a3 = round(row['Conversion_Share_3'] / nsd, 2)
        others = round((100 - (row['Conversion_Share_1'] + row['Conversion_Share_2'] + row['Conversion_Share_3'])) / nsd, 2)
        values =[]
        # Apply delta rounding
        for val in [a1, a2, a3, others]:
            if val - np.floor(val) >= DELTA:
                val = np.ceil(val).astype(int)
            else:
                val = np.floor(val).astype(int)
            values.append(val)    
        
        total_orders = sum(values)
        values.append(total_orders)
        # Створюємо новий рядок у вигляді DataFrame
        new_row = pd.DataFrame([values], columns=columns)
        # Додаємо новий рядок до results_df за допомогою concat
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    return results_df


def _prepare_search_volume_samples(data: pd.DataFrame) -> pd.DataFrame:
    """Підготовлює search volume приклади."""
    # Створюємо порожній DataFrame для результатів
    result_samples = pd.DataFrame(columns=['Search_Term', 'totalOrders', 'conversion', 'searchVolume'])
    
    for _, row in data.iterrows():
        search_term = row['Search_Term']
        total_orders = row['totalOrders']
        conversions = row['conversions']
        
        for orders, conversion in zip(total_orders, conversions):
            search_volume = round(orders / conversion)
            new_row = pd.DataFrame([[search_term, orders, conversion, search_volume]], 
                                  columns=['Search_Term', 'totalOrders', 'conversion', 'searchVolume'])
            result_samples = pd.concat([result_samples, new_row], ignore_index=True)
    #result_samples.sort_values('searchVolume', ascending=False)
    # Сортуємо за searchVolume за спаданням
    return result_samples

def prepare_data_samples(df: pd.DataFrame, count_total_orders: int = 5) -> pd.DataFrame:
    """Готує можливі приклади з можливими замовленнями, conversion, and search volume."""
    # Створюємо порожній DataFrame для результатів
    prepared_data = pd.DataFrame(columns=['Search_Term', 'samples', 'totalOrders', 'conversions'])
    
    for _, row in df.iterrows():
        samples = calculate_sampling_orders(row)
        if samples.empty:
            continue
        
        # Calculate possible conversions
        word_count = len(row['Search_Term'].split())
        total_orders = samples['totalOrders'].head(count_total_orders).tolist()
        conversions = []
        # Обчислюємо searchVolume та conversion
           
        if row['Conversion_Share_1'] != 0:
            z = 'a1'
            y = row['Conversion_Share_1']
            x = row['Click_Share_1']
        elif row['Conversion_Share_2'] != 0:
            z = 'a2'
            y = row['Conversion_Share_2']
            x = row['Click_Share_2']
        elif row['Conversion_Share_3'] != 0:
            z = 'a3'
            y = row['Conversion_Share_3']
            x = row['Click_Share_3'] 

        for i, orders in enumerate(total_orders):
            click_volume = (samples.iloc[i][z] / y) * 100
            search_volume = (click_volume/ x) * 100
            conversion = round(orders / search_volume, 6) if orders != 0 else 0
            if word_count > 1 or (word_count == 1 and conversion < 0.02):
                conversions.append(conversion)
        
        # Додаємо новий рядок до DataFrame
        new_row = pd.DataFrame([[row['Search_Term'], samples, total_orders, conversions]], 
                              columns=['Search_Term', 'samples', 'totalOrders', 'conversions'])
        prepared_data = pd.concat([prepared_data, new_row], ignore_index=True)
    
    return _prepare_search_volume_samples(prepared_data)
    

    

In [369]:
processed_data = prepare_data_samples(new_df)
processed_data.head()

C:\Users\user\AppData\Local\Temp\ipykernel_11424\3379768576.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_samples = pd.concat([result_samples, new_row], ignore_index=True)


,Search_Term,totalOrders,conversion,searchVolume
0,christmas decorations,185,0.052893,3498
1,christmas decorations,922,0.052721,17488
2,christmas decorations,1659,0.052702,31479
3,christmas decorations,2395,0.052673,45469
4,christmas decorations,3132,0.052674,59460


In [370]:
processed_data.tail(10)

,Search_Term,totalOrders,conversion,searchVolume
1255,ps4 controller,106,0.154449,686
1256,ps4 controller,520,0.160072,3249
1257,ps4 controller,936,0.159822,5857
1258,ps4 controller,1351,0.160475,8419
1259,ps4 controller,1768,0.160338,11027
1260,kitchen gadgets,525,0.086502,6069
1261,kitchen gadgets,2618,0.087821,29811
1262,kitchen gadgets,4711,0.087970,53552
1263,kitchen gadgets,6805,0.087838,77472
1264,kitchen gadgets,8898,0.087913,101214


побудуємо функцію для сортування даних по першому унікальному Search Term

In [181]:

def sort_by_first_search_volume(data: pd.DataFrame) -> pd.DataFrame:
    """
    Сортує дані за першим значенням searchVolume для кожного Search_Term,
    зберігаючи всі рядки, пов’язані з кожним Search_Term, разом.
    """
    # Групуємо дані за Search_Term
    grouped = data.groupby('Search_Term', sort=False)

    # Знаходимо перше значення searchVolume для кожної групи
    first_search_volume = grouped['searchVolume'].first().reset_index()

    # Сортуємо групи за першим значенням searchVolume (за спаданням)
    sorted_groups = first_search_volume.sort_values(by='searchVolume', ascending=False)

    # Об'єднуємо відсортовані групи, зберігаючи порядок
    sorted_data = pd.concat([grouped.get_group(name) for name in sorted_groups['Search_Term']])

    # Скидаємо індекс для кращої читабельності
    sorted_data = sorted_data.reset_index(drop=True)

    return sorted_data

In [182]:
sorted_data = sort_by_first_search_volume(processed_data)
sorted_data.head(10)

,Search_Term,totalOrders,conversion,searchVolume
0,sweaters for women,8418,0.013472,624852
1,sweaters for women,42088,0.013471,3124341
2,sweaters for women,75759,0.013471,5623859
3,sweaters for women,109429,0.013471,8123302
4,sweaters for women,143099,0.013471,10622745
5,long sleeve shirts for women,6342,0.015026,422068
6,long sleeve shirts for women,31708,0.015003,2113444
7,long sleeve shirts for women,57072,0.015012,3801759
8,long sleeve shirts for women,82437,0.015016,5489944
9,long sleeve shirts for women,107801,0.015018,7178120


Побудуємо функцію для пошуку даних з файлу за вказаними пошуковими термінами

In [211]:


def get_ba_data_from_file(file_path: str, search_terms: list) -> pd.DataFrame:
    """
    Читає дані з Excel файлу та обробляє їх для вказаних пошукових термінів.
    Знаходить рядки, де Search Term містить будь-яке слово зі списку search_terms.
    
    Параметри:
    -----------
    file_path : str
        Шлях до Excel файлу
    search_terms : list
        Список термінів для пошуку в стовпці Search Term
    
    Повертає:
    --------
    pd.DataFrame
        Відфільтрований DataFrame з обчисленими конверсіями
    """
    # Читаємо Excel файл
    df = pd.read_excel(file_path,skiprows=1, header=0)
    #переіменовуємо стовпці
    new_columns = ['Department', 'Search_Term', 'Search_Frequency_Rank', 'ASIN_1',
       'Product_Title_1', 'Click_Share_1', 'Conversion_Share_1',
       'ASIN_2', 'Product_Title_2', 'Click_Share_2', 'Conversion_Share_2',
        'ASIN_3', 'Product_Title_3', 'Click_Share_3', 'Conversion_Share_3',]
    df.columns = new_columns
    # Створюємо маску для часткових збігів
    mask = df['Search_Term'].str.lower().apply(
        lambda x: any(term.lower() in x for term in search_terms)
    )
    
    # Фільтруємо датафрейм і створюємо копію
    result_df = df[mask].copy()
    
    # Обчислюємо конверсії (множимо значення на 100)
    result_df['Conversion_Share_1'] = result_df['Conversion_Share_1'] * 100
    result_df['Conversion_Share_2'] = result_df['Conversion_Share_2'] * 100
    result_df['Conversion_Share_3'] = result_df['Conversion_Share_3'] * 100
    result_df['Click_Share_1'] = result_df['Click_Share_1'] * 100
    result_df['Click_Share_2'] = result_df['Click_Share_2'] * 100
    result_df['Click_Share_3'] = result_df['Click_Share_3'] * 100
    
    # Додаємо порядковий номер на основі першого знайденого терміну(опціонально)
    def get_matching_term_index(search_term):
        search_term_lower = search_term.lower()
        for idx, term in enumerate(search_terms):
            if term.lower() in search_term_lower:
                return idx
        return len(search_terms)  # значення за замовчуванням
        
    result_df['order'] = result_df['Search_Term'].apply(get_matching_term_index)
    
    # Сортуємо за порядковим номером
    result_df = result_df.sort_values('order')
    
    return result_df



In [212]:
# Приклад використання:
search_terms = ['christmas', 'sweater', 'boots', 'iphone']
file_path = r'C:\Users\user\Downloads\brandAnalytics\data7days\13.xls'
df_search = get_ba_data_from_file(file_path, search_terms)
df_search.head()

,Department,Search_Term,Search_Frequency_Rank,ASIN_1,Product_Title_1,Click_Share_1,Conversion_Share_1,ASIN_2,Product_Title_2,Click_Share_2,Conversion_Share_2,ASIN_3,Product_Title_3,Click_Share_3,Conversion_Share_3,order
0,Amazon.com,christmas decorations,1,B08B5PHMH6,Ivenf Christmas Decorations Outdoor Yard Front...,5.2653,5.4300,B0B652CTLV,318 PCS 12 Sheets Christmas Snowflake Window C...,3.9585,6.5093,B07HHBQJGJ,LUDILO 135Pcs Christmas Window Clings Snowflak...,3.7396,7.5548,0
62,Amazon.com,christmas ornaments,63,B09Z2LHJ46,Sea Team Multi-Size Shatterproof Clear Plastic...,5.1034,4.4018,B0B2LCVFVB,Antteez Colored Christmas Tree Balls Ornaments...,4.2832,9.2551,B0BKZZNMX5,Barry Santa Wood Christmas Ornament Funny Acry...,3.9460,3.9503,0
63,Amazon.com,christmas decorations indoor,64,B0771MJHKT,Desktop Miniature Pine Tree Tabletop Christmas...,4.8280,8.3218,B07KPMPNHF,JOYIN 5 Pieces Christmas Kitchen Appliance Han...,4.6819,5.5479,B0B978BKRR,"YEAHOME Christmas Decorations Indoor, 24”/2FT ...",3.4407,2.4965,0
66,Amazon.com,family christmas pjs matching sets,67,B09XFZTWM3,The Children's Place Family Matching Christmas...,9.6264,12.7527,B093WBFJMW,IFFEI Matching Family Pajamas Sets Christmas P...,8.1336,5.7543,B08KQ7KFTR,Family Christmas Pjs Matching Sets Baby Christ...,4.5994,6.8429,0
73,Amazon.com,christmas tree skirt,74,B07JK4MXFX,Dremisland 36 Luxury Faux Fur Christmas Tree S...,7.5184,7.2187,B08DR9NFMC,Christmas Tree Skirt - 36 inches White Christm...,5.7340,6.9002,B0BCZ1YXM6,"Funux Christmas Tree Skirt, 48 Inch White Faux...",3.8147,3.4501,0


In [176]:
df_search.shape
#знайдено 67 позицій по даних термінах

(67, 16)

In [177]:

def clearBadData(data: pd.DataFrame, window_size: int = 7) -> pd.DataFrame:
    """
    Очищає дані від  outliers використовуючи різні підходи.
    """
    # Копіюємо дані для очищення
    cleaned_data = data.copy()
    
    # Видалення викидів за стандартним відхиленням для 'totalOrders'
    for col in ['totalOrders']:
        # Крок 1: Видалити дані, які відрізняються від середнього більш ніж на 3 стандартних відхилення
        n_std = 3
        mean = cleaned_data[col].mean()
        std = cleaned_data[col].std()
        cleaned_data = cleaned_data[abs(cleaned_data[col] - mean) <= n_std * std]
        
        # Крок 2: Видалити дані, які відрізняються від нового середнього більш ніж на 2.5 стандартних відхилення
        n_std = 2.5
        mean = cleaned_data[col].mean()
        std = cleaned_data[col].std()
        cleaned_data = cleaned_data[abs(cleaned_data[col] - mean) <= n_std * std]
        
        # Крок 3: Видалити дані, які відрізняються від нового середнього більш ніж на 1 стандартне відхилення
        n_std = 1
        mean = cleaned_data[col].mean()
        std = cleaned_data[col].std()
        cleaned_data = cleaned_data[abs(cleaned_data[col] - mean) <= n_std * std]
    
    # Видалення викидів за відсотковим відхиленням для 'conversion'
    for col in ['conversion']:
        # Крок 1: Видалити дані, які відрізняються від середнього більш ніж на 30%
        percent = 0.3
        mean = cleaned_data[col].mean()
        cleaned_data = cleaned_data[abs(cleaned_data[col] - mean) <= mean * percent]
        
        # Крок 2: Видалити дані, які відрізняються від нового середнього більш ніж на 20%
        percent = 0.2
        mean = cleaned_data[col].mean()
        cleaned_data = cleaned_data[abs(cleaned_data[col] - mean) <= mean * percent]
        
        # Крок 3: Видалити дані, які відрізняються від нового середнього більш ніж на 10%
        percent = 0.1
        mean = cleaned_data[col].mean()
        cleaned_data = cleaned_data[abs(cleaned_data[col] - mean) <= mean * percent]
    
    return cleaned_data

In [183]:
cleaned_df = clearBadData(sorted_data)
print("Cleaned DataFrame:")
cleaned_df

Cleaned DataFrame:


,Search_Term,totalOrders,conversion,searchVolume
140,nike shoes men,1520,0.056158,27066
255,new balance womens shoes,830,0.057180,14516
256,new balance womens shoes,4151,0.057194,72578
260,kitchen organization,874,0.062016,14093
261,kitchen organization,4370,0.062016,70466
...,...,...,...,...
925,dresser for bedroom,142,0.064749,2193
926,dresser for bedroom,704,0.067077,10495
927,dresser for bedroom,1266,0.067348,18798
928,dresser for bedroom,1830,0.067139,27257


ЧАСТИНА 2
Виходячи з даних які ми отримуємо з сайту, в нас дані відсортовані по "- search_frequency_rank - його порядкового номеру в списку серчтермів по популярності" по спаданню тобто перший search term має найбільшу популярність по кількості запитів (чому саме по кількості запитів а не замовлень: в даних є search terms де всі конверсії по трьом асінам рівні нулю тобто замовлень взагалі не було то в рейтинг вони могли попасти тільки по запитам).
Тому шукаючи варіанти можливих total orders and searchVolume ми маємо враховувати що починаючи з кінця кожен наступний searchVolume має бути більшим за попередній тому далі напишемо функцію підбору замовлень та кількості запитів які будуть виконувати дану умову

Щоб не писати цілий ряд операцій огорнемо в  функцію  отримання даних з файлу та провіримо її

In [4]:

import os

def process_excel_data(file_path):
    """
    Зчитує Excel-файл, перейменовує стовпці та обчислює конверсії.

    Параметри:
        file_path (str): Шлях до Excel-файлу.

    Повертає:
        pd.DataFrame: Оброблений DataFrame.

    Викидає:
        FileNotFoundError: Якщо файл не знайдено.
        ValueError: Якщо виникає помилка при зчитуванні файлу.
    """
    # Перевірка наявності файлу
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл {file_path} не знайдено.")
    
    try:
        # Зчитуємо дані з Excel-файлу
        df = pd.read_excel(file_path, skiprows=1, header=0)
    except Exception as e:
        raise ValueError(f"Помилка при зчитуванні файлу: {e}")
    
    # Перейменовуємо стовпці
    new_columns = [
        'Department', 'Search_Term', 'Search_Frequency_Rank', 'ASIN_1',
        'Product_Title_1', 'Click_Share_1', 'Conversion_Share_1',
        'ASIN_2', 'Product_Title_2', 'Click_Share_2', 'Conversion_Share_2',
        'ASIN_3', 'Product_Title_3', 'Click_Share_3', 'Conversion_Share_3'
    ]
    df.columns = new_columns
    
    # Обчислюємо конверсії (множимо значення на 100)
    df['Conversion_Share_1'] = df['Conversion_Share_1'] * 100
    df['Conversion_Share_2'] = df['Conversion_Share_2'] * 100
    df['Conversion_Share_3'] = df['Conversion_Share_3'] * 100
    df['Click_Share_1'] = df['Click_Share_1'] * 100
    df['Click_Share_2'] = df['Click_Share_2'] * 100
    df['Click_Share_3'] = df['Click_Share_3'] * 100
    
    return df

In [5]:
file_path1 = r'C:\Users\user\Downloads\brandAnalytics\data7days\13.xls'
df = process_excel_data(file_path1)
df.head()

,Department,Search_Term,Search_Frequency_Rank,ASIN_1,Product_Title_1,Click_Share_1,Conversion_Share_1,ASIN_2,Product_Title_2,Click_Share_2,Conversion_Share_2,ASIN_3,Product_Title_3,Click_Share_3,Conversion_Share_3
0,Amazon.com,christmas decorations,1,B08B5PHMH6,Ivenf Christmas Decorations Outdoor Yard Front...,5.2653,5.4300,B0B652CTLV,318 PCS 12 Sheets Christmas Snowflake Window C...,3.9585,6.5093,B07HHBQJGJ,LUDILO 135Pcs Christmas Window Clings Snowflak...,3.7396,7.5548
1,Amazon.com,thanksgiving outfit women,2,B09FXHL254,SheIn Women's V Neck Sleeveless Pocket Corduro...,4.5553,1.9108,B0BFWXHD2F,Gobble Funny Thanksgiving T-Shirt Women Turkey...,1.3965,2.1231,B0BFWVCM64,Gobble Funny Thanksgiving T-Shirt Women Turkey...,1.1762,1.5924
2,Amazon.com,yellowstone,3,B0B8T22BYG,Yellowstone: Inside the Ranch,37.6087,64.6818,B0B8QQK2TG,Yellowstone Season 5: Pts 1 & 2,12.2555,3.6922,B07DWNHLFG,Daybreak,7.6955,5.8172
3,Amazon.com,yellowstone season 5,4,B0B8T22BYG,Yellowstone: Inside the Ranch,47.0374,74.0830,B0B8QQK2TG,Yellowstone Season 5: Pts 1 & 2,18.3375,5.1684,B0B8Q96Q59,Yellowstone Season 5: Pts 1 & 2,14.7028,8.3781
4,Amazon.com,christmas tree,5,B09F6GYVZL,Yaheetech 4.5ft Pre-lit Artificial Christmas T...,4.2566,2.8793,B08J7JRJL3,Best Choice Products 6ft Pre-Lit Pre-Decorated...,4.1603,5.2102,B0BHTTZ9H9,Best Choice Products 4.5ft Pre-Lit Snow Flocke...,3.8411,3.7934


In [6]:

new_df = df.copy()

Алгоритм такий: починаємо роботу з останнього рядка так як в даному search_term кількість запитів найменша.
В циклі підбираємо кількості замовлень по кожному асіну та потім загальну та загальну кількість запитів search_volume1 та порівнюємо його з попереднім (початкову ми задали 0) Якщо умова виконується то ми ці дані записуємо в результуючий датафрейм переприсвоюємо Search_Term та виходимо з циклу. Далі переходимо до наступного Search_Term по зростанню  .Якщо всі конверсії по трьом асінам рівні 0 ми пропускаємо даний рядок. В кінці ми сортуємо датафрейм по спаданню

In [9]:

import numpy as np

def process_data(df: pd.DataFrame, DELTA=0.05) -> pd.DataFrame:
    """
    Обробляє DataFrame, починаючи з останнього рядка, і обчислює searchVolume та conversion.
    Зупиняється, коли searchVolume поточного рядка стає більшим за попередній.
    """
    # Сортуємо DataFrame за Search_Frequency_Rank за зростанням (останній рядок перший)
    df = df.sort_values('Search_Frequency_Rank', ascending=False).reset_index(drop=True)
    
    # Створюємо порожній DataFrame для результатів
    result_df = pd.DataFrame(columns=['Search_Term', 'totalOrders', 'conversion', 'searchVolume'])
    
    # Ініціалізуємо попередній searchVolume як 0
    prev_search_volume = 0
    
    for _, row in df.iterrows():
        conversions = list(filter(None, [row['Conversion_Share_1'], row['Conversion_Share_2'], row['Conversion_Share_3']]))
        # Перевіряємо, чи список порожній
        if not conversions:
            continue  # Переходимо до наступної ітерації
        # Обчислюємо min_conversion
        min_conversion = min(conversions)
        # Обчислюємо можливі замовлення
        i=2
        while True:
            nsd = min_conversion / i
            a1 = round(row['Conversion_Share_1'] / nsd, 2)
            a2 = round(row['Conversion_Share_2'] / nsd, 2)
            a3 = round(row['Conversion_Share_3'] / nsd, 2)
            others = round((100 - (row['Conversion_Share_1'] + row['Conversion_Share_2'] + row['Conversion_Share_3'])) / nsd, 2)
            values = []
            # застосовуємо delta округлення
            for val in [a1, a2, a3, others]:
                if val - np.floor(val) >= DELTA:
                    val = np.ceil(val).astype(int)
                else:
                    val = np.floor(val).astype(int)
                values.append(val)

            total_orders = sum(values)
            values.append(total_orders)
            # Обчислюємо word_count
            word_count = len(row['Search_Term'].split())
            i+=2
            # Обчислюємо searchVolume та conversion
            if row['Conversion_Share_1'] != 0:
                click_volume_1 = (values[0]/ row['Conversion_Share_1']) * 100
                search_volume1 = round((click_volume_1 / row['Click_Share_1']) * 100)
            elif  row['Conversion_Share_2'] != 0:
                click_volume_1 = (values[1]/ row['Conversion_Share_2']) * 100
                search_volume1 = round((click_volume_1 / row['Click_Share_2']) * 100) 
            elif  row['Conversion_Share_3'] != 0:
                click_volume_1 = (values[2]/ row['Conversion_Share_3']) * 100
                search_volume1 = round((click_volume_1 / row['Click_Share_3']) * 100) 
            else:
                break
            
            # Перевіряємо, чи search_volume1 більший за попередній searchVolume
            if search_volume1 >= prev_search_volume:
                # Обчислюємо conversion
                
                orders = values[-1]
                conversion = round(orders/ search_volume1, 6) if orders != 0 else 0
                if word_count > 1 or (word_count == 1 and conversion < 0.02):
                    # Додаємо результат до result_df
                    new_row = pd.DataFrame([[row['Search_Term'], orders, conversion, search_volume1]], 
                                          columns=['Search_Term', 'totalOrders', 'conversion', 'searchVolume'])
                    result_df = pd.concat([result_df, new_row], ignore_index=True)
                    # Оновлюємо попередній searchVolume
                    prev_search_volume = search_volume1
                break
                
    # Сортуємо результат за searchVolume за спаданням та скидаємо індекс
    result_df = result_df.sort_values('searchVolume', ascending=False).reset_index(drop=True)
    return result_df

In [10]:
process_df = process_data(new_df)
print(process_df.shape)
process_df.head(10)

C:\Users\user\AppData\Local\Temp\ipykernel_3364\1517886777.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, new_row], ignore_index=True)


(253, 4)


,Search_Term,totalOrders,conversion,searchVolume
0,christmas decorations,19376,0.052659,367953
1,thanksgiving outfit women,16706,0.045442,367636
2,yellowstone season 5,172744,0.470378,367245
3,christmas tree,15630,0.042569,367166
4,black friday deals 2022,27494,0.075150,365853
5,sweaters for women,8418,0.023171,363301
6,womens fall fashion 2022,5104,0.016241,314262
7,winter coats for women,8787,0.029531,297548
8,fleece lined leggings women,21685,0.073700,294233
9,sweater dress for women,5030,0.017121,293798


In [11]:

process_df.tail()

,Search_Term,totalOrders,conversion,searchVolume
248,stocking holders for mantle,769,0.119540,6433
249,puffer vest women,474,0.078738,6020
250,long sleeve bodysuit for women,119,0.023891,4981
251,ps4 controller,210,0.158252,1327
252,kitchen gadgets,106,0.084800,1250


якщо в нас є дані по кількості за вказаний період хоча б по одному з seach terms ми можемо знайти відношення між цими даними та даними з таблиці та помножити на значення всі інші значення таблиці

In [15]:

def multiply_and_round(n, df):
    # Копіюємо DataFrame, щоб уникнути змін у вихідному DataFrame
    result_df = df.copy()
    
    # Помножуємо та округляємо значення в стовпцях
    result_df['totalOrders'] = (result_df['totalOrders'] * n).round().astype(int)
    result_df['searchVolume'] = (result_df['searchVolume'] * n).round().astype(int)
    
    return result_df

# Виводимо результат
res_df = multiply_and_round(1.5, process_df)
res_df.head()

,Search_Term,totalOrders,conversion,searchVolume
0,christmas decorations,29064,0.052659,551929
1,thanksgiving outfit women,25059,0.045442,551454
2,yellowstone season 5,259116,0.470378,550867
3,christmas tree,23445,0.042569,550749
4,black friday deals 2022,41241,0.075150,548779


In [16]:
res_df.tail(10)

,Search_Term,totalOrders,conversion,searchVolume
243,fuzzy socks,553,0.044901,12327
244,record player,1981,0.169337,11701
245,nail polish,597,0.051401,11614
246,star trek,2985,0.271561,10992
247,christmas wrapping paper,705,0.064828,10875
248,stocking holders for mantle,1153,0.119540,9649
249,puffer vest women,711,0.078738,9030
250,long sleeve bodysuit for women,178,0.023891,7471
251,ps4 controller,315,0.158252,1990
252,kitchen gadgets,159,0.084800,1875


Побудуємо таблицю sorted_df, в якій дані будуть відсортовані по кількості замовлень по спаданню

In [17]:
# Сортування за стовпцем totalOrders по спаданню
sorted_df = res_df.sort_values(by='totalOrders', ascending=False)

sorted_df.head(10)

,Search_Term,totalOrders,conversion,searchVolume
2,yellowstone season 5,259116,0.470378,550867
21,nana hats for bananas,167568,0.418345,400549
31,gifts for women,167053,0.426321,391849
36,yellowstone season 1,135991,0.349134,389511
101,kids tablet,126808,0.565580,224209
127,xbox series x,112863,0.673216,167647
111,playstation 5,104886,0.531148,197470
50,paw patrol,88734,0.257838,344146
15,nintendo switch,78117,0.190141,410838
42,air fryer,71325,0.185445,384615


В подальшому ми можемо будувати датасет по певному Search_Term збираючи інформацію про totalOrders(кількість замовлень)  за кожен день ..Зібравши їх достатню кількість (за рік два) зможемо робити прогнозування часових рядів(Це процес аналізу та прогнозування майбутніх значень на основі історичних даних, які збираються через рівні проміжки часу) тобто зможемо прогнозувати кількість замовлень по певному виду товарів за допомогою машиного навчання.

пошук даних за списком ключових слів що можуть міститись в search terms

In [18]:

def filter_by_search_terms(res_df: pd.DataFrame, search_terms: list) -> pd.DataFrame:
    """
    Фільтрує DataFrame, залишаючи рядки, де стовпець 'Search_Term' містить одне зі слів у search_terms.

    Параметри:
        res_df (pd.DataFrame): Вхідний DataFrame.
        search_terms (list): Список слів для пошуку.

    Повертає:
        pd.DataFrame: Відфільтрований DataFrame.
    """
    # Створюємо регулярний вираз для пошуку будь-якого зі слів
    pattern = '|'.join(search_terms)
    
    # Знаходимо рядки, де Search_Term містить одне зі слів
    filtered_df = res_df[res_df['Search_Term'].str.contains(pattern, case=False, regex=True)]
    
    return filtered_df

In [19]:
# Список слів для пошуку
search_terms = ['christmas', 'sweater', 'boots', 'iphone']

# Виклик функції
filtered_df = filter_by_search_terms(res_df, search_terms)

# Виводимо результат
print(filtered_df.shape)
filtered_df.head(10)

(62, 4)


,Search_Term,totalOrders,conversion,searchVolume
0,christmas decorations,29064,0.052659,551929
3,christmas tree,23445,0.042569,550749
5,sweaters for women,12627,0.023171,544951
9,sweater dress for women,7545,0.017121,440697
10,christmas lights,39060,0.090644,430918
11,boots for women,20890,0.048653,429379
12,christmas pajamas for family,33762,0.079915,422472
14,ugly christmas sweater for women,19084,0.045803,416664
16,ugly christmas sweater men,54693,0.133131,410820
17,christmas lights outdoor,61857,0.150725,410395


побудуємо функцію для збереження таблиці в форматі Excel  та провіримо її (для її виконання потрібно встановити модуль openpyxl)

In [20]:

def save_to_xls(df, file_name, sheet_name='Sheet1'):
    """
    Зберігає DataFrame у файл формату .xls (Excel).

    Параметри:
    df (pd.DataFrame): DataFrame, який потрібно зберегти.
    file_name (str): Назва файлу (наприклад, 'output.xls').
    sheet_name (str): Назва аркуша в Excel (за замовчуванням 'Sheet1').
    """
    try:
        # Зберігаємо DataFrame у файл .xls
        df.to_excel(file_name, sheet_name=sheet_name, index=False)
        print(f"Файл '{file_name}' успішно збережено!")
    except Exception as e:
        print(f"Помилка при збереженні файлу: {e}")



In [21]:

# Зберігаємо DataFrame у файл
save_to_xls(res_df, 'output_13_12_2022.xlsx', sheet_name='SearchData')

Файл 'output_13_12_2022.xlsx' успішно збережено!


In [22]:
df_search = pd.read_excel('output_13_12_2022.xlsx')
df_search.head()

,Search_Term,totalOrders,conversion,searchVolume
0,christmas decorations,29064,0.052659,551929
1,thanksgiving outfit women,25059,0.045442,551454
2,yellowstone season 5,259116,0.470378,550867
3,christmas tree,23445,0.042569,550749
4,black friday deals 2022,41241,0.075150,548779


Радий буду діалогу по даному підходу. Далі можна розглядати фільтрацію та ставити різні обмеження. 
А дані функції можна записувати в окремі python(.py) файли щоб потім їх можна було б використовувати в php для роботи з користувачами .Ну і звичайно прогнозування на основі попередніх даних....